# Tabular Time Series Related API Demo with NOAA Weather Data

Copyright (c) Microsoft Corporation. All rights reserved. <br>
Licensed under the MIT License.

In this notebook, you will learn how to use the Tabular Time Series related API to filter the data by time windows for sample data uploaded to Azure blob storage. 

The detailed APIs to be demoed in this script are:
- Create Tabular Dataset instance
- Assign fine timestamp column and coarse timestamp column for Tabular Dataset to activate Time Series related APIs
- Clear fine timestamp column and coarse timestamp column
- Filter in data before a specific time
- Filter in data after a specific time
- Filter in data in a specific time range
- Filter in data for recent time range

Besides above APIs, you'll also see:
- Create and load a Workspace
- Load National Oceanic & Atmospheric (NOAA) weather data into Azure blob storage
- Create and register NOAA weather data as a Tabular dataset
- Re-load Tabular Dataset from your Workspace

## Import Dependencies

If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, run the cells below to install the Azure Machine Learning Python SDK and create an Azure ML Workspace that's required for this demo.

## Prepare Environment

Print out your version of the Azure ML Python SDK. Version 1.0.60 or above is required for TabularDataset with timeseries attribute. 

In [ ]:
import azureml.core
azureml.data.__version__

## Import Packages

In [ ]:
# import packages
import os

import pandas as pd

from calendar import monthrange
from datetime import datetime, timedelta

from azureml.core import Dataset, Datastore, Workspace, Run
from azureml.opendatasets import NoaaIsdWeather

## Set up Configuraton and Create Azure ML Workspace

If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, go through the [configuration notebook](https://github.com/Azure/MachineLearningNotebooks/blob/master/configuration.ipynb) first if you haven't already to establish your connection to the Azure ML Workspace.

In [ ]:
ws     = Workspace.from_config()
dstore = ws.get_default_datastore()

dset_name = 'weather-data-florida'

print(ws.name, ws.resource_group, ws.location, ws.subscription_id, dstore.name, sep = '\n')

## Load Data to Blob Storage

This demo uses public NOAA weather data. You can replace this data with your own. The first cell below creates a Pandas Dataframe object with the first 6 months of 2019 NOAA weather data. The last cell saves the data to a CSV file and uploads the CSV file to Azure blob storage to the location specified in the datapath variable. Currently, the Dataset class only reads uploaded files from blob storage. 

**NOTE:** to reduce the size of data, we will only keep specific rows with a given stationName.

In [ ]:
target_years = [2019]

for year in target_years:
    for month in range(1, 12+1):
        path = 'data/{}/{:02d}/'.format(year, month)
        
        try:  
            start = datetime(year, month, 1)
            end   = datetime(year, month, monthrange(year, month)[1]) + timedelta(days=1)
            isd   = NoaaIsdWeather(start, end).to_pandas_dataframe()
            isd   = isd[isd['stationName'].str.contains('FLORIDA', regex=True, na=False)]
            
            os.makedirs(path, exist_ok=True)
            isd.to_parquet(path + 'data.parquet')
        except Exception as e:
            print('Month {} in year {} likely has no data.\n'.format(month, year))
            print('Exception: {}'.format(e))

Upload data to blob storage so it can be used as a Dataset.

In [ ]:
dstore.upload('data', dset_name, overwrite=True, show_progress=True)

## Create & Register Tabular Dataset with time-series trait from Blob

The API on Tabular datasets with time-series trait is specially designed to handle Tabular time-series data and time related operations more efficiently. By registering your time-series dataset, you are publishing your dataset to your workspace so that it is accessible to anyone with the same subscription id. 

Create Tabular Dataset instance from blob storage datapath.

**TIP:** you can set virtual columns in the partition_format. I.e. if you partition the weather data by state and city, the path can be '/{STATE}/{CITY}/{coarse_time:yyy/MM}/data.parquet'. STATE and CITY would then appear as virtual columns in the dataset, allowing for efficient filtering by these grains. 

In [ ]:
datastore_path = [(dstore, dset_name + '/*/*/data.parquet')]
dataset        = Dataset.Tabular.from_parquet_files(path=datastore_path, partition_format = dset_name + '/{coarse_time:yyyy/MM}/data.parquet')

Assign fine timestamp column for Tabular Dataset to activate Time Series related APIs. The column to be assigned should be a Date type, otherwise the assigning will fail.

In [ ]:
# for this demo, leave out coarse_time so fine_grain_timestamp is used
tsd = dataset.with_timestamp_columns(fine_grain_timestamp='datetime') # , coarse_grain_timestamp='coarse_time')

Register the dataset for easy access from anywhere in Azure ML and to keep track of versions, lineage. 

In [ ]:
# register dataset to Workspace
registered_ds = tsd.register(ws, dset_name, create_new_version=True, description='Data for Tabular Dataset with time-series trait demo.', tags={ 'type': 'TabularDataset' })

## Reload the Dataset from Workspace

In [ ]:
# get dataset by dataset name
tsd = Dataset.get_by_name(ws, name=dset_name)
tsd.to_pandas_dataframe().head(5)

## Filter Data by Time Windows

Once your data has been loaded into the notebook, you can query by time using the time_before(), time_after(), time_between(), and time_recent() functions. You can also choose to drop or keep certain columns.  

### Before Time Input

In [ ]:
# select data that occurs before a specified date
tsd2 = tsd.time_before(datetime(2019, 6, 12))
tsd2.to_pandas_dataframe().tail(5)

## After Time Input

In [ ]:
# select data that occurs after a specified date
tsd2 = tsd.time_after(datetime(2019, 5, 30))
tsd2.to_pandas_dataframe().head(5)

## Before & After Time Inputs

You can chain time functions together.

**NOTE:** You must set the coarse_grain_timestamp to None to filter on the fine_grain_timestamp. The below cell will fail unless the second line is uncommented 

In [ ]:
# select data that occurs within a given time range
#tsd  = tsd.with_timestamp_columns(fine_grain_timestamp='datetime', coarse_grain_timestamp=None)
tsd2 = tsd.time_after(datetime(2019, 1, 2)).time_before(datetime(2019, 1, 10))
tsd2.to_pandas_dataframe().head(5)

### Time Range Input

In [ ]:
# another way to select data that occurs within a given time range
tsd2 = tsd.time_between(start_time=datetime(2019, 1, 31, 23, 59, 59), end_time=datetime(2019, 2, 7))
tsd2.to_pandas_dataframe().head(5)

## Time Recent Input

This function takes in a datetime.timedelta and returns a dataset containing the data from datetime.now()-timedelta() to datetime.now().

In [ ]:
tsd2 = tsd.time_recent(timedelta(weeks=5, days=0))
tsd2.to_pandas_dataframe().head(5)

**NOTE:** This will return an empty dataframe there is no data within the last 2 days.

In [ ]:
tsd2 = tsd.time_recent(timedelta(days=2))
tsd2.to_pandas_dataframe().tail(5)

### Drop Columns

<font color=red>The columns to be dropped should NOT include timstamp columns.</font><br>Below operation will lead to exception.

In [ ]:
try:
    tsd2 = tsd.drop_columns(columns=['snowDepth', 'version', 'datetime'])
except Exception as e:
    print('Expected exception : {}'.format(str(e)))

Drop will succeed if modify column list to exclude timestamp columns.

In [ ]:
tsd2 = tsd.drop_columns(columns=['snowDepth', 'version', 'upload_date'])
tsd2.take(5).to_pandas_dataframe().sort_values(by='datetime')

### Keep Columns

<font color=red>The columns to be kept should ALWAYS include timstamp columns.</font><br>Below operation will lead to exception.

In [ ]:
try:
    tsd2 = tsd.keep_columns(columns=['snowDepth'], validate=False)
except Exception as e:
    print('Expected exception : {}'.format(str(e)))

Keep will succeed if modify column list to include timestamp columns.

In [ ]:
tsd2 = tsd.keep_columns(columns=['snowDepth', 'datetime', 'coarse_time'], validate=False)
tsd2.to_pandas_dataframe().tail()

## Resetting Timestamp Columns

Rules for reseting are:
- You cannot assign 'None' to fine_grain_timestamp while assign a valid column name to coarse_grain_timestamp because coarse_grain_timestamp is optional while fine_grain_timestamp is mandatory for Tabular time series data.
- If you assign 'None' to fine_grain_timestamp, then both fine_grain_timestamp and coarse_grain_timestamp will all be cleared.
- If you assign only 'None' to coarse_grain_timestamp, then only coarse_grain_timestamp will be cleared.

In [ ]:
# Illegal clearing, exception is expected.
try:
    tsd2 = tsd.with_timestamp_columns(fine_grain_timestamp=None, coarse_grain_timestamp='coarse_time')
except Exception as e:
    print('Cleaning not allowed because {}'.format(str(e)))

# clear both
tsd2 = tsd.with_timestamp_columns(fine_grain_timestamp=None, coarse_grain_timestamp=None)
print('after clean both with None/None, timestamp columns are: {}'.format(tsd2.timestamp_columns))

# clear coarse_grain_timestamp only and assign 'datetime' as fine timestamp column
tsd2 = tsd2.with_timestamp_columns(fine_grain_timestamp='datetime', coarse_grain_timestamp=None)
print('after clean coarse timestamp column, timestamp columns are: {}'.format(tsd2.timestamp_columns))

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/work-with-data/datasets/datasets-tutorial/datasets-tutorial.png)